In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np
import matplotlib.pyplot as plt


LOG.propagate = False

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

In [ ]:
# DRIVE AT WALL
incoming_data = []
time = []
distance = []
speed = []
data = []
def wall_drive(uiud, byte_arr):
    # variables
    global incoming_data
    global data
    global time
    global distance
    global speed
    
    # handle incoming data
    incoming_data = ble.bytearray_to_string(byte_arr)
    data.append(incoming_data)
    
    #indexes
    time_start = incoming_data.index('T')+2
    time_end = incoming_data.index('S')-1
    speed_start = time_end + 3
    speed_end = incoming_data.index('D')-1
    distance_start = speed_end + 3
    
    # create new arrays
    time.append(float(incoming_data[time_start:time_end]))
    speed.append(float(incoming_data[speed_start:speed_end]))
    distance.append(float(incoming_data[distance_start:]))
    

   
ble.start_notify(ble.uuid['RX_STRING'], wall_drive)
ble.send_command(CMD.DRIVE_AT_WALL, "")



In [ ]:
#PLOT DATA

# # Scale distances down
# for idx in range(len(distance)):
#         distance[idx] = distance[idx]/1000 
        
# Make velocity data
velocity = [0]
velocity_time = []
smooth = 0
for idx in range(len(time)):
    if(idx > 0 and smooth == 3):
        velocity.append((distance[idx] - distance[idx-2]) / (time[idx] - time[idx - 2])) 
        velocity_time.append(time[idx])
        smooth = 0
    smooth += 1

print(time)
print(distance)
print(velocity)

#Distance 
plt.plot (time[1:len(time)-2], distance[1:len(distance)-2])
plt.xticks(np.arange(min(time), max(time), 750))
plt.yticks(np.arange(min(distance), max(distance), 100))

plt.title ('Distance vs Time')
plt.xlabel ('Time')
plt.ylabel ('Distance')
plt.show()

# # Speed
# plt.plot (time [0:], speed [0:])

# plt.title ('Speed vs Time')
# plt.xlabel ('Time')
# plt.ylabel ('Speed')
# plt.show()

# Velocity
plt.plot (velocity_time [0:], velocity [0:])
plt.title ('Velocity vs Time')
plt.xlabel ('Time')
plt.ylabel ('Velocity')
plt.show()


ble.stop_notify(ble.uuid['RX_STRING'])




In [ ]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [ ]:
ble.disconnect()